In [1]:
import os

In [2]:
%pwd

'd:\\Vedaant\\MachineLearningTutorial\\Chicken-Disease-Classification\\Chicken-Disease-Classification\\research'

In [3]:
os.chdir('../')

In [40]:
%pwd

'd:\\Vedaant\\MachineLearningTutorial\\Chicken-Disease-Classification\\Chicken-Disease-Classification'

In [41]:
import tensorflow as tf

model = tf.keras.models.load_model('artifacts/training/model.h5')

from dataclasses import dataclass
from pathlib import Path

c

In [42]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories, save_json

In [59]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_validation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/Chicken-fecal-images",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [60]:
from urllib.parse import urlparse

In [61]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )

    
    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)

    
    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

In [62]:
try:
    config = ConfigurationManager()
    val_config = config.get_validation_config()
    evaluation = Evaluation(val_config)
    evaluation.evaluation()
    evaluation.save_score()
except Exception as e:
    raise e

[2025-01-29 08:54:02,857: INFO: common: Successfully loaded YAML file: config\config.yaml]
[2025-01-29 08:54:02,860: INFO: common: Successfully loaded YAML file: params.yaml]
[2025-01-29 08:54:02,861: INFO: common: Created directory: artifacts]
Found 116 images belonging to 2 classes.
8/8 [==============================] - 21s 2s/step - loss: 36.6945 - accuracy: 0.5000
[2025-01-29 08:54:24,690: INFO: common: Saved JSON data to: scores.json]
